In [1]:
import os

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import plotly.express as px

from sklearn import preprocessing
from sklearn.cluster import KMeans

In [2]:
# Get PATH folder from previous directorys
def get_path(prev_folders:int=0):
    for i in range(prev_folders-1): os.chdir('../')  # Change to previous folder
    PATH = os.path.dirname(os.getcwd()) + '/'
    PATH = PATH.replace('\\', '/')
    return PATH

# Environment settings

In [3]:
pd.options.display.max_columns = None  # Remove "dots" from display when printing dataframes
PATH = get_path(prev_folders=2)

Adjunto encontrara el archivo “202306_Prueba_CD_prob_acuaticos.parquet”. En este documento se presenta información de trabajadores (afiliado cotizante) y beneficiarios de la caja de compensación familiar Compensar con la siguiente información:
- Id (identificación)
- fuente (indica si el registro corresponde a un trabajador (afiliado) o un beneficiario)
- parentesco_beneficiario (indica la relación que tiene el beneficiario con el trabajador cotizante)
- PeriodoVinculación (fecha de vinculación a la caja)
- genero
- nivel_socioeconomico
- Categoria
- descripcion_familia (tipo de familia)
- edad
- Educación (indica si la persona ha tomado servicios de educación)
- Recreación (indica si la persona ha tomado servicios de recreación)
- Acondicionamiento Físico (indica si la persona ha tomado servicios de acondicionamiento físico)
- Motricidad (indica si la persona ha tomado servicios de motricidad)
- Deportes Terrestres (indica si la persona ha tomado servicios de deportes terrestres)
- Recompra (indica si la persona ha tomado servicios de deportes acuáticos en años anteriores)
- Objetivo (indica si la persona ha tomado servicios de deportes acuáticos en el último año)

# Read data

In [4]:
df = pd.read_parquet(PATH + 'data/202306_Prueba_CD_prob_acuaticos.parquet')

# Preprocesing

In [5]:
df = df.rename({
    'PeriodoVinculación':'periodo_vinculacion',
    'Categoria':'categoria',
    'Educación':'educacion',
    'Recreación':'recreacion',
    'Acondicionamiento Físico':'acondicionamiento_fisico',
    'Motricidad':'motricidad',
    'Deportes Terrestres':'deportes_terrestres',
    'Recompra':'recompra',
    'Objetivo':'objetivo'
    }, axis=1
    )

df['parentesco_beneficiario'] = df['parentesco_beneficiario'].fillna('Ninguno')
df['periodo_vinculacion'] = pd.to_datetime(df['periodo_vinculacion'], format='%Y-%m-%d')

# Descriptive analysis

a. Realice un análisis descriptivo de la información presentada.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        1000 non-null   int32         
 1   fuente                    1000 non-null   object        
 2   parentesco_beneficiario   1000 non-null   object        
 3   periodo_vinculacion       1000 non-null   datetime64[ns]
 4   genero                    1000 non-null   object        
 5   nivel_socioeconomico      1000 non-null   int32         
 6   categoria                 1000 non-null   object        
 7   descripcion_familia       787 non-null    object        
 8   edad                      1000 non-null   int32         
 9   educacion                 1000 non-null   object        
 10  recreacion                1000 non-null   int32         
 11  acondicionamiento_fisico  1000 non-null   int32         
 12  motricidad           

In [7]:
df.describe()

,id,nivel_socioeconomico,edad,recreacion,acondicionamiento_fisico,motricidad,deportes_terrestres,recompra,objetivo
count,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.00000
mean,5.095164e+06,1.782000,32.567000,0.027000,0.026000,0.01200,0.036000,0.091000,0.10000
std,2.924070e+06,1.728428,19.367979,0.162164,0.159215,0.10894,0.186383,0.287753,0.30015
min,1.154190e+05,-1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
25%,2.439276e+06,0.000000,17.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
50%,5.137560e+06,2.000000,30.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
75%,7.717380e+06,3.000000,45.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
max,9.994874e+06,6.000000,101.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000


In [8]:
df.isnull().sum()

id                            0
fuente                        0
parentesco_beneficiario       0
periodo_vinculacion           0
genero                        0
nivel_socioeconomico          0
categoria                     0
descripcion_familia         213
edad                          0
educacion                     0
recreacion                    0
acondicionamiento_fisico      0
motricidad                    0
deportes_terrestres           0
recompra                      0
objetivo                      0
dtype: int64

In [9]:
df.sample(n=15).head(15)

,id,fuente,parentesco_beneficiario,periodo_vinculacion,genero,nivel_socioeconomico,categoria,descripcion_familia,edad,educacion,recreacion,acondicionamiento_fisico,motricidad,deportes_terrestres,recompra,objetivo
325,3293377,Beneficiario,Hijos,2019-04-01,Masculino,5,A,MonoParental,22,NO,0,0,0,0,0,0
858,2247803,Afiliado,Ninguno,2023-02-08,Femenino,3,A,None,25,NO,0,0,0,0,0,0
847,3238276,Afiliado,Ninguno,2022-05-10,Femenino,3,A,Unipersonal,31,NO,0,0,0,0,0,0
704,5290572,Afiliado,Ninguno,2023-04-01,Femenino,4,C,None,48,NO,0,0,0,0,0,0
380,2044722,Afiliado,Ninguno,2022-01-03,Femenino,0,A,Extensas,39,NO,0,0,0,1,0,0
745,9070878,Beneficiario,Hijos,2022-01-17,Masculino,0,B,Extensas,3,NO,1,0,0,0,0,0
519,5214880,Afiliado,Ninguno,2023-01-26,Masculino,2,A,None,35,NO,0,0,0,0,0,0
93,9547560,Afiliado,Ninguno,2023-03-15,Masculino,4,A,None,45,NO,0,0,0,0,0,0
827,2690422,Beneficiario,Hijos,2021-02-01,Masculino,4,A,MonoParental,17,NO,0,0,0,0,0,0
21,9572454,Afiliado,Ninguno,2021-09-25,Masculino,0,A,Extensas,35,NO,0,0,0,0,0,0


# Identification model for water sports

b. Desarrolle un modelo que permita identificar clientes potenciales para adquirir servicios de deportes acuáticos.

## Imputation Hot-Deck (variation)

In [10]:
label_encoder = preprocessing.LabelEncoder()

df['ENC_fuente'] = label_encoder.fit_transform(df['fuente'])
df['ENC_parentesco_beneficiario'] = label_encoder.fit_transform(df['parentesco_beneficiario'])
df['ENC_categoria'] = label_encoder.fit_transform(df['categoria'])

In [11]:
X = df[[
    'ENC_fuente', 'ENC_parentesco_beneficiario',
    'nivel_socioeconomico',
    'ENC_categoria'
    ]].values  # Variables related to the category to be predicted

# Clustering
kmeans_model = KMeans(n_clusters=4)
kmeans_model = kmeans_model.fit(X)
labels = kmeans_model.predict(X)
centroids = kmeans_model.cluster_centers_  # centroids values

for i in range(0, len(df)):
    role, relation, socio_lev, category = df.loc[i, [
        'ENC_fuente', 'ENC_parentesco_beneficiario',
        'nivel_socioeconomico', 'ENC_categoria'
        ]] # variables as an array
    X_new = np.array([[role, relation, socio_lev, category]])
    new_label = kmeans_model.predict(X_new)
    df.at[i, 'PRED_descripcion_familia'] = new_label

c:\Users\almontao\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\almontao\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\almontao\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


# Models

## Ordinary Least Squares regression (OLS)

In [12]:
regresion = smf.ols('objetivo ~ fuente + parentesco_beneficiario + genero + nivel_socioeconomico + categoria + descripcion_familia + recreacion + acondicionamiento_fisico + motricidad + recompra',
              data=df
              )
results = regresion.fit()

# print(res.rsquared)
# print(res.params)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               objetivo   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.495
Method:                 Least Squares   F-statistic:                     43.81
Date:                Sun, 29 Oct 2023   Prob (F-statistic):          8.76e-105
Time:                        15:27:55   Log-Likelihood:                 26.570
No. Observations:                 787   AIC:                            -15.14
Df Residuals:                     768   BIC:                             73.56
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

## 